In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from dataset import load_dataset, get_formatted_questions
from model import ModelConfig, get_completion
from answer_evaluator import get_final_selection

from dotenv import load_dotenv
import logging

In [2]:
logging.basicConfig(level=logging.INFO)
load_dotenv()

True

In [3]:
print('Model used -- ')
print(ModelConfig())

Model used -- 
ModelConfig(
    model='deepseek-ai/DeepSeek-V3',
    max_tokens=1000,
    temperature=0.1,
    top_p=0.1,
    base_url='https://router.huggingface.co/together'
)


In [4]:
data = load_dataset()
problems = get_formatted_questions(data, sample_size=10)


INFO:root:Loading dataset Supa-AI/STEM-en-ms with language 'en' and split 'eval'
INFO:root:Dataset loaded. Shape: (614, 9)
INFO:root:Filtering out questions with figures
INFO:root:Filtered dataset shape: (322, 9)


In [5]:
trace = {}

In [60]:

for id_, question, correct_answer in problems:
    if id_ not in [560] and id_ in trace:
        print(f'SKIPPING {id_}')
        continue
    model_response = get_completion(question)
    model_answer, reasoning = get_final_selection(model_response)
    trace[id_] = {
        'id': id_,
        'question': question,
        'correct_answer': correct_answer,
        'model_response': model_response,
        'model_answer': model_answer,
        'judge_reasoning': reasoning
    }

SKIPPING 498
SKIPPING 187
SKIPPING 312
SKIPPING 114


INFO:httpx:HTTP Request: POST https://router.huggingface.co/together/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completion successful with tokens 440
INFO:httpx:HTTP Request: POST https://router.huggingface.co/hf-inference/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completion successful with tokens 822


SKIPPING 34
SKIPPING 493
SKIPPING 110
SKIPPING 287
SKIPPING 267


In [61]:
score = 0
total = len(trace)

mistakes = []
for id_, row in trace.items():
    correct_answer = row['correct_answer']
    model_answer = row['model_answer']

    got_correct = correct_answer == model_answer
    score += int(got_correct)

    if not got_correct:
        mistakes.append(row)
        break

accuracy = score / total
print(accuracy)

0.4
